# NLP Code Along

For this code along we will build a spam filter! We'll use the various NLP tools we learned about as well as a new classifier, Naive Bayes.

We'll use a classic dataset for this - UCI Repository SMS Spam Detection: https://archive.ics.uci.edu/ml/datasets/SMS+Spam+Collection

In [4]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('nlp').getOrCreate()

In [6]:
data = spark.read.csv('smsspamcollection/SMSSpamCollection', inferSchema=True,sep='\t')

In [7]:
data.head()

Row(_c0='ham', _c1='Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...')

In [10]:
data = data.withColumnRenamed('_c0', 'class').withColumnRenamed('_c1', 'text')

In [12]:
data.show()

+-----+--------------------+
|class|                text|
+-----+--------------------+
|  ham|Go until jurong p...|
|  ham|Ok lar... Joking ...|
| spam|Free entry in 2 a...|
|  ham|U dun say so earl...|
|  ham|Nah I don't think...|
| spam|FreeMsg Hey there...|
|  ham|Even my brother i...|
|  ham|As per your reque...|
| spam|WINNER!! As a val...|
| spam|Had your mobile 1...|
|  ham|I'm gonna be home...|
| spam|SIX chances to wi...|
| spam|URGENT! You have ...|
|  ham|I've been searchi...|
|  ham|I HAVE A DATE ON ...|
| spam|XXXMobileMovieClu...|
|  ham|Oh k...i'm watchi...|
|  ham|Eh u remember how...|
|  ham|Fine if thats th...|
| spam|England v Macedon...|
+-----+--------------------+
only showing top 20 rows



## Clean and Prepare the Data

**Create a new length feature:**

In [13]:
from pyspark.sql.functions import length

In [14]:
data = data.withColumn('length', length('text'))

In [17]:
data.show()

+-----+--------------------+------+
|class|                text|length|
+-----+--------------------+------+
|  ham|Go until jurong p...|   111|
|  ham|Ok lar... Joking ...|    29|
| spam|Free entry in 2 a...|   155|
|  ham|U dun say so earl...|    49|
|  ham|Nah I don't think...|    61|
| spam|FreeMsg Hey there...|   147|
|  ham|Even my brother i...|    77|
|  ham|As per your reque...|   160|
| spam|WINNER!! As a val...|   157|
| spam|Had your mobile 1...|   154|
|  ham|I'm gonna be home...|   109|
| spam|SIX chances to wi...|   136|
| spam|URGENT! You have ...|   155|
|  ham|I've been searchi...|   196|
|  ham|I HAVE A DATE ON ...|    35|
| spam|XXXMobileMovieClu...|   149|
|  ham|Oh k...i'm watchi...|    26|
|  ham|Eh u remember how...|    81|
|  ham|Fine if thats th...|    56|
| spam|England v Macedon...|   155|
+-----+--------------------+------+
only showing top 20 rows



In [19]:
# Pretty clear difference

data.groupBy('class').mean().show()

+-----+-----------------+
|class|      avg(length)|
+-----+-----------------+
|  ham|71.45431945307645|
| spam|138.6706827309237|
+-----+-----------------+



## Feature Transformations

In [21]:
from pyspark.ml.feature import (Tokenizer, StopWordsRemover, CountVectorizer,
                                IDF, StringIndexer)

In [22]:
tokenizer = Tokenizer(inputCol='text', outputCol='token_text')
stop_remover = StopWordsRemover(inputCol='token_text', outputCol='stop_tokens')
count_vec = CountVectorizer(inputCol='stop_tokens', outputCol='c_vec')
idf = IDF(inputCol='c_vec', outputCol='tf_idf')
ham_spam_to_num = StringIndexer(inputCol='class', outputCol='label')

In [23]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.linalg import Vector

In [24]:
clean_up = VectorAssembler(inputCols=['tf_idf', 'length'], outputCol='features')

### The Model

We'll use Naive Bayes, but feel free to play around with this choice!

In [25]:
from pyspark.ml.classification import NaiveBayes

In [26]:
# Use defaults
nb = NaiveBayes()

### Pipeline

In [27]:
from pyspark.ml import Pipeline

In [29]:
data_prep_pipe = Pipeline(stages=[ham_spam_to_num,
                                  tokenizer,
                                  stop_remover,
                                  count_vec,
                                  idf,
                                  clean_up])

In [30]:
cleaner = data_prep_pipe.fit(data)

In [31]:
clean_data = cleaner.transform(data)

### Training and Evaluation!

In [34]:
clean_data = clean_data.select(['label','features'])

In [35]:
clean_data.show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(13424,[7,11,31,6...|
|  0.0|(13424,[0,24,297,...|
|  1.0|(13424,[2,13,19,3...|
|  0.0|(13424,[0,70,80,1...|
|  0.0|(13424,[36,134,31...|
|  1.0|(13424,[10,60,139...|
|  0.0|(13424,[10,53,103...|
|  0.0|(13424,[125,184,4...|
|  1.0|(13424,[1,47,118,...|
|  1.0|(13424,[0,1,13,27...|
|  0.0|(13424,[18,43,120...|
|  1.0|(13424,[8,17,37,8...|
|  1.0|(13424,[13,30,47,...|
|  0.0|(13424,[39,96,217...|
|  0.0|(13424,[552,1697,...|
|  1.0|(13424,[30,109,11...|
|  0.0|(13424,[82,214,47...|
|  0.0|(13424,[0,2,49,13...|
|  0.0|(13424,[0,74,105,...|
|  1.0|(13424,[4,30,33,5...|
+-----+--------------------+
only showing top 20 rows



In [36]:
(training, testing) = clean_data.randomSplit([0.7,0.3])

In [37]:
spam_predictor = nb.fit(training)

In [38]:
data.printSchema()

root
 |-- class: string (nullable = true)
 |-- text: string (nullable = true)
 |-- length: integer (nullable = true)



In [39]:
test_results = spam_predictor.transform(testing)

In [40]:
test_results.show()

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(13424,[0,1,2,41,...|[-1060.5816182348...|[1.0,4.1267902148...|       0.0|
|  0.0|(13424,[0,1,5,15,...|[-1003.3469668552...|[1.0,3.4371219216...|       0.0|
|  0.0|(13424,[0,1,5,20,...|[-805.24189988291...|[1.0,4.3093971753...|       0.0|
|  0.0|(13424,[0,1,7,8,1...|[-1194.5431431238...|[1.0,5.2130706753...|       0.0|
|  0.0|(13424,[0,1,7,15,...|[-656.58961135735...|[1.0,4.3935431870...|       0.0|
|  0.0|(13424,[0,1,11,32...|[-882.49771726612...|[1.0,2.7437078056...|       0.0|
|  0.0|(13424,[0,1,14,18...|[-1371.1603703494...|[1.0,2.2189618517...|       0.0|
|  0.0|(13424,[0,1,15,20...|[-675.79642885056...|[1.0,1.1221348871...|       0.0|
|  0.0|(13424,[0,1,23,63...|[-1312.3402043650...|[1.0,1.4235722693...|       0.0|
|  0.0|(13424,[0

In [41]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [42]:
acc_eval = MulticlassClassificationEvaluator()
acc = acc_eval.evaluate(test_results)
print('Accuracy of model at predicting spam: {}'.format(acc))

Accuracy of model at predicting spam: 0.9203015961022222


Not bad considering we're using straight math on text data! Try switching out the classification models! Or even try to come up with other engineered features!

## Great Job!